# Flight Delay Time Statistics

###  This project focuses on developing an interactive dashboard to visualize and analyze flight delay statistics using Python. Leveraging libraries such as Dash, Plotly, and Jupyter-Dash, the application enables dynamic visualizations for understanding delay patterns across various airlines and delay types. By embedding the dashboard within a Jupyter Notebook, this project highlights the ease of integrating interactive data applications into a data scientist's workflow, providing an accessible and engaging way to explore data insights.

## Install jupyter-dash:

In [2]:
pip install jupyter-dash


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Importing Required Libraries :

##### pandas: Handles data loading and manipulation.
##### jupyter_dash: Allows the Dash app to run inside a Jupyter Notebook.
##### dash: Used for creating the layout and interactive components of the dashboard.
##### plotly.express: A library to create interactive plots like line charts.

In [3]:
import pandas as pd
from jupyter_dash import JupyterDash  # Import JupyterDash for embedding Dash apps in notebooks
from dash import dcc, html  # Dash components for creating layouts and interactivity
from dash.dependencies import Input, Output  # For callback functionality
import plotly.express as px  # For creating interactive visualizations


In [4]:
# Read the airline data into a pandas dataframe
airline_data = pd.read_csv(
    'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/airline_data.csv', 
    encoding="ISO-8859-1",
    dtype={'Div1Airport': str, 'Div1TailNum': str, 
           'Div2Airport': str, 'Div2TailNum': str}
)


In [12]:
airline_data.head()

,Unnamed: 0,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,DOT_ID_Reporting_Airline,IATA_CODE_Reporting_Airline,...,Div4WheelsOff,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum
0,1295781,1998,2,4,2,4,1998-04-02,AS,19930,AS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1125375,2013,2,5,13,1,2013-05-13,EV,20366,EV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,118824,1993,3,9,25,6,1993-09-25,UA,19977,UA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,634825,1994,4,11,12,6,1994-11-12,HP,19991,HP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1888125,2017,3,8,17,4,2017-08-17,UA,19977,UA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Creating the JupyterDash App

In [5]:
# Create a JupyterDash application
app = JupyterDash(__name__)

C:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\dash\dash.py:579: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



##### JupyterDash: Initializes a Dash application that can run directly inside a Jupyter Notebook.

## Designing the Dashboard Layout

In [6]:
# Build dash app layout
app.layout = html.Div(children=[
    html.H1(
        'Flight Delay Time Statistics', 
        style={'textAlign': 'center', 'color': '#503D36', 'font-size': 30}
    ),
    html.Div(
        ["Input Year: ", 
         dcc.Input(
             id='input-year', 
             value='2010', 
             type='number', 
             style={'height': '35px', 'font-size': 30}
         )], 
        style={'font-size': 30}
    ),
    html.Br(),
    html.Br(),
    # Segment 1
    html.Div([
        html.Div(dcc.Graph(id='carrier-plot')),
        html.Div(dcc.Graph(id='weather-plot'))
    ], style={'display': 'flex'}),
    # Segment 2
    html.Div([
        html.Div(dcc.Graph(id='nas-plot')),
        html.Div(dcc.Graph(id='security-plot'))
    ], style={'display': 'flex'}),
    # Segment 3
    html.Div(dcc.Graph(id='late-plot'), style={'width': '65%'})
])


Layout Structure:
- html.Div: A container for organizing layout components.
- html.H1: Displays the title at the top of the dashboard.
- dcc.Input: A textbox for users to input the year.
- dcc.Graph: Placeholder for charts to be dynamically updated.
  
Styling:
- Custom CSS styles (e.g., textAlign, font-size, etc.) enhance the dashboard’s appearance.
  
Segments:
- The layout divides the plots into three sections for better readability.

## Function to Compute Delays

In [7]:
# Function to compute info
def compute_info(airline_data, entered_year):
    # Select data for the given year
    df = airline_data[airline_data['Year'] == int(entered_year)]
    # Compute delay averages
    avg_car = df.groupby(['Month', 'Reporting_Airline'])['CarrierDelay'].mean().reset_index()
    avg_weather = df.groupby(['Month', 'Reporting_Airline'])['WeatherDelay'].mean().reset_index()
    avg_NAS = df.groupby(['Month', 'Reporting_Airline'])['NASDelay'].mean().reset_index()
    avg_sec = df.groupby(['Month', 'Reporting_Airline'])['SecurityDelay'].mean().reset_index()
    avg_late = df.groupby(['Month', 'Reporting_Airline'])['LateAircraftDelay'].mean().reset_index()
    return avg_car, avg_weather, avg_NAS, avg_sec, avg_late


This helper function:
1. Filters the dataset by the input year.
2. Groups the data by Month and Reporting Airline.
3. Calculates the average delay for different types (carrier, weather, etc.).
4. Returns the computed averages for plotting.

## Adding Callbacks to Update Graphs

In [8]:
# Callback to update figures
@app.callback(
    [
        Output('carrier-plot', 'figure'),
        Output('weather-plot', 'figure'),
        Output('nas-plot', 'figure'),
        Output('security-plot', 'figure'),
        Output('late-plot', 'figure')
    ],
    Input('input-year', 'value')
)
def update_graphs(entered_year):
    # Compute the required data
    avg_car, avg_weather, avg_NAS, avg_sec, avg_late = compute_info(airline_data, entered_year)
    
    # Create figures
    carrier_fig = px.line(avg_car, x='Month', y='CarrierDelay', color='Reporting_Airline', 
                          title='Average Carrier Delay Time (Minutes) by Airline')
    weather_fig = px.line(avg_weather, x='Month', y='WeatherDelay', color='Reporting_Airline', 
                          title='Average Weather Delay Time (Minutes) by Airline')
    nas_fig = px.line(avg_NAS, x='Month', y='NASDelay', color='Reporting_Airline', 
                      title='Average NAS Delay Time (Minutes) by Airline')
    sec_fig = px.line(avg_sec, x='Month', y='SecurityDelay', color='Reporting_Airline', 
                      title='Average Security Delay Time (Minutes) by Airline')
    late_fig = px.line(avg_late, x='Month', y='LateAircraftDelay', color='Reporting_Airline', 
                       title='Average Late Aircraft Delay Time (Minutes) by Airline')
    
    return [carrier_fig, weather_fig, nas_fig, sec_fig, late_fig]


- Inputs: Listens for changes in the input-year field.
- Outputs: Updates the graphs (dcc.Graph) dynamically.
- Uses plotly.express.line to create line plots for each delay type.

## Running the App

In [9]:
# Run the app in the notebook
app.run_server(mode='inline')


## Conclusion :

####   The Flight Delay Time Statistics dashboard effectively demonstrates how data visualization can provide insights into airline performance metrics, specifically focusing on flight delays due to various reasons such as carrier delays, weather conditions, NAS (National Aviation System) issues, security, and late aircraft arrivals. By leveraging Dash and Plotly, the project presents interactive, visually appealing graphs that allow users to analyze delays across different airlines and months.